# Yandex Music (big cities music)
**(comparing of users Moscow and St. Petersburg musical preferences)**

**Problem statement**

We need to compare the behavior (musical preferences) of the two capitals users based on the data of Yandex Music.

**The purpose of the research** - to test three hypotheses:
1. The activity of users depends on the day of the week. And in Moscow and St. Petersburg it manifests itself differently.
2. Monday morning in Moscow some genres prevail, and in Petersburg - others. Also on Friday evening, different genres prevail - depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research progress**

Data on user behavior will be obtained from the `yandex_music_project.csv` file. Nothing is known about the data quality. Therefore, before testing hypotheses, a review of the data is required.

We'll check the data for errors and evaluate their impact on the research. Then, at the preprocessing stage, we'll find an opportunity to correct the most critical data errors.
 
Thus, the research will take place in three stages:
1. Data review.
2. Data preprocessing.
3. Hypothesis testing.

## Data review

Let's make the first idea about Yandex Music data.

**Import required libraries**

Import `pandas` library to work with data tables (dataframes).

In [1]:
# pandas library import
import pandas as pd

**Reading a file (dataset) with data**

Read the `yandex_music_project.csv` file from the `\datasets` folder and save it in the `df` variable:

In [2]:
# reading data file and saving to df
df = pd.read_csv('datasets\yandex_music_project.csv')

**Displaying a table fragment for preliminary analysis**

Let's display the first ten rows of the table:

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


**General information about the table and its contents**


Get general information about the `df` table:

In [4]:
# getting general information about the df table data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


The `df` dataframe has seven columns. The data type in all columns is `object`.

According to the data documentation:
* `  userID` - user ID;
* `Track` — track name;
* `artist` — artist name;
* `genre` — genre name;
* `  City  ` - user's city;
* `time` - start time of listening;
* `Day` is the day of the week.

The number of values in the columns varies. This means there are missing values in the data.

**Table column names style violations**

The following style violations appear in the column names of the `df` table:
* Lowercase letters are combined with uppercase.
* There are spaces.
* Missing underscore `_` to write names in "snake_case".
* Instead of transliteration, a literal translation of names is used (it is not a violation of style, but we note it as a feature).

**Conclusions**

Each line of the table contains data about the listened track. Some of the columns describe the composition itself: title, artist and genre. The rest of the data tells us about the user: what city he is from, when he listened to music.

Preliminarily, it can be argued that the data are sufficient to test the hypotheses. But there are missing values in the data, and discrepancies with good style in the columns names.

To move forward, we need to fix problems in the data.

## Data preprocessing

Data preprocessing in this project can be done in three simple steps:
- fix the style in the table columns headers;
- eliminate missing values in the data;
- check the data for duplicates.

### Heading style

**Displaying the `df` table column names**

Let's display the column names:

In [5]:
# list of df table column names
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

**Correction of column names**


Let's bring the names in line with good style:
* write a few words in the title in the “snake_case”,
* make all characters lowercase,
* eliminate spaces.

To do this, rename the columns like this:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
# rename column names
df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}, inplace=True)

**Test display of table `df` column names**


Let's check the result. To do this, once again display the columns names:

In [7]:
# check results - list column names
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values

**Counting missing values in dataframe data**

First, let's calculate how many missing values are in the table:

In [8]:
# missing values count
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values affect the research. So, in `track` and `artist`, gaps are not important for our work. It suffices to replace them with explicit notation.

But missing values in `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. In practice, it would be correct to determine the cause of the gaps and restore the data. This option is not available in this research. So, we have to:
* fill in these gaps with explicit designations;
* estimate how much they will damage the calculations.

**Fill missing values in the table**

Let's replace the missing values in the `track`, `artist` and `genre` columns with the string `'unknown'`. To do this, create a `columns_to_replace` list, iterate through its elements with a `for` loop, and for each column, replace the missing values:

In [9]:
# loop through column names and replace missing values with 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')    

**Checking for missing values in a table**

Make sure there are no gaps in the table. To do this, let's count the missing values again.

In [10]:
# counting missing values
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates

**Counting explicit duplicates in a table**

Let's count explicit duplicates in the table:

In [11]:
# counting explicit duplicates
df.duplicated().sum()

3826

**Remove explicit duplicates**

Let's call a special `pandas` method to remove explicit duplicates:

In [12]:
# removing explicit duplicates
df = df.drop_duplicates()

**Check count of explicit duplicates in a table**

Once again, let's count the explicit duplicates in the table - make sure that we completely get rid of them:

In [13]:
# check for duplicates
df.duplicated().sum()

0

Now let's get rid of the implicit duplicates in the `genre` column. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the research.

**Displaying a list of unique genre names**

Let's display a list of unique genre names, sorted alphabetically. For this:
1. extract the desired dataframe column;
2. apply the sorting method to it;
3. for the sorted column, call a method that will return unique values from the column.

In [14]:
# view unique genre names
genres = df['genre']
genres.sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Clear table from implicit duplicates**

Let's go through the list and find implicit duplicates of music genre names. These may be misspelled titles or alternative titles in the same genre.

Based on visual search results, the following implicit duplicates of the `hiphop` genre were found:
* *hip*,
* *hop*,
* *hip-hop*.

To clear them from the table, we use the `replace()` method with two arguments: a list of duplicate strings (including *hip*, *hop*, and *hip-hop*) and a string with the correct value. It is necessary to fix the `genre` column in the `df` table: replace each value from the list of duplicates with the correct one. Instead of `hip`, `hop` and `hip-hop` the table should have the value `hiphop`:

In [15]:
# eliminate implicit duplicates
df = df.replace(to_replace=['hip', 'hop', 'hip-hop'], value='hiphop')

**Checking for no implicit duplicates in a table**

Let's check that we have replaced the wrong names:

* hip,
* hop,
* hip-hop.

To do this, display a sorted list of unique values in the `genre` column:

In [16]:
# checking for implicit duplicates
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Conclusions**

Data preprocessing found three problems in the data:

- headline style violations,
- missing values,
- duplicates - explicit and implicit.

We've fixed the headers to make the table easier to work with. Without duplicates, the research will become more accurate.

Missing values have been replaced with `'unknown'`. It remains to be seen whether the gaps in the `genre` column will harm the research.

Now we can move on to hypothesis testing.

## Testing hypotheses

### Comparison of user behavior in two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Let's check this assumption using the data of three days of the week - Monday, Wednesday and Friday. For this:

* Let's separate the users of Moscow and St. Petersburg.
* Compare how many tracks each group of users listened to on Monday, Wednesday and Friday.

**Assessment of activity in listening to music by residents of two cities**

First, we will perform each of the calculations separately.

Let's evaluate the activity of users in each city. Let's group the data by city and count the listening in each group.


In [17]:
# Counting listening in each city
df.groupby(by='city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

There are more listening in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

**Evaluation of user activity depending on the day of the week**

Now let's group the data by day of the week and count the listening on Monday, Wednesday, and Friday. Let's take into account that the data contains information about listening only for these days.

In [18]:
# listening count for each of the three days
df.groupby(by='day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

**Group by city and day of the week**

Earlier, we grouped the original table separately by the city of listening to Yandex Music and by day of the week.
Let's combine these two calculations - grouping by city and days of the week - into one.

To do this, we will create a `number_tracks()` function that will count the listening for a given day and city. It needs two parameters:
* day of the week,
* city name.

In the function, we save the rows of the source table into a variable, which have the value:
  * in the `day` column is equal to the `day` parameter,
  * in the `city` column is equal to the `city` parameter.

Then we count the values in the `user_id` column of the resulting table. The result will be saved to a new variable and returned from the function.

In [19]:
# <create number_tracks() function>
# A function is declared with two parameters: day, city.
# The track_list variable stores those rows of the df table for which
# the value in the 'day' column is equal to the day parameter and at the same time the value
# in the 'city' column is equal to the city parameter.
# The track_list_count variable stores the number of values in the 'user_id' column,
# calculated by the count() method for the track_list table.
# The function returns a number - the value of track_list_count.

# Function for counting listening for a specific city and day.
# Using sequential filtering with logical indexing, it
# first get the rows with the desired day from the source table,
# then from the result it will filter the rows with the desired city,
# the count() method will count the number of values in the user_id column.
# This is the number the function will return as a result

def number_tracks(day, city):
    track_list = df[(df.day == day) & (df.city == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

**Retrieve listening data by city and day of the week**

Let's call `number_tracks()` six times, changing the value of the parameters so that we get data for each city on each of the three days.

In [20]:
# number of listening in Moscow on Mondays
number_tracks('Monday', 'Moscow')

15740

In [21]:
# number of listening in St. Petersburg on Mondays
number_tracks('Monday', 'Saint-Petersburg')

5614

In [22]:
# number of listening in Moscow on Wednesdays
number_tracks('Wednesday', 'Moscow')

11056

In [23]:
# number of listening in St. Petersburg on Wednesdays
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [24]:
# number of listening in Moscow on Fridays
number_tracks('Friday', 'Moscow')

15945

In [25]:
# number of listening in St. Petersburg on Fridays
number_tracks('Friday', 'Saint-Petersburg')

5895

**Create a table with music listening data by city and day of the week**

Let's create a table using the `pd.DataFrame` constructor, where
* column names - `['city', 'monday', 'wednesday', 'friday']`;
* data is the results we got above with the `number_tracks` function.

In [26]:
# table with results
pd.DataFrame(data={'city': ['Moscow', 'Saint_Petersburg'], 
                   'monday':    [15740, 5614],
                   'wednesday': [11056, 7003],
                   'friday':    [15945, 5895]})

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint_Petersburg,5614,7003,5895


**Conclusions**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

This means that the data speaks in favor of the first hypothesis - users listen to music differently in Moscow and St. Petersburg depending on the day of the week.

### Music at the beginning and end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.

**Preparing data for analyzing the genre of music listened to depending on the day of the week**

Let's save tables with data in two variables:
* for Moscow - in `moscow_general`;
* for St. Petersburg - in `spb_general`.

In [27]:
# getting the moscow_general table from those rows of the df table,
# for which the value in the 'city' column is 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [28]:
# getting the spb_general table from those rows of the df table,
# for which the value in the 'city' column is 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

**The function of analyzing the top 10 listening genres on a given day and time**

Let's create a `genre_weekday()` function with four parameters:
* table (dataframe) with data,
* day of the week,
* start timestamp in 'hh:mm' format,
* last timestamp in 'hh:mm' format.

The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [29]:
# Declaring the function genre_weekday() with parameters table, day, time1, time2.
# This function returns information about the most popular genres on the specified day in
# given time:
# 1) the genre_df variable saves those rows of the transmitted dataframe table, for
#    of which at the same time:
#    - the value in the day column is equal to the value of the day argument
#    - the value in the time column is greater than the value of the time1 argument
#    - the value in the time column is less than the value of the time2 argument
#    Use sequential filtering with boolean indexing.
# 2) group dataframe genre_df by genre column, take one of its
#    columns and use the count() method to count the number of entries for each
#    of present genres, write the resulting Series to a variable
#    genre_df_count
# 3) sort genre_df_count in descending order of occurrence and save
#    into the genre_df_sorted variable
# 4) return a Series of the first 10 genre_df_sorted values, these will be the top 10
#    popular genres (on the specified day, at the specified time)

def genre_weekday(df, day, time1, time2):
    # sequential filtering
    # leave in genre_df only those df lines whose day is equal to day
    genre_df = df[df['day'] == day] 
    # leave in genre_df only those genre_df lines that have a time less than time2
    genre_df = genre_df[genre_df['time'] < time2] 
    # leave in genre_df only those genre_df lines that have a time greater than time1
    genre_df = genre_df[genre_df['time'] > time1] 
    # group the filtered dataframe by a column with genre names, take the genre column and count the number of rows for each genre using the count() method
    genre_df_grouped = genre_df.groupby(by='genre')['genre'].count() 
    # sort the result in descending order (so that the most popular genres are at the beginning of the Series)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    # return a Series with the 10 most popular genres in the specified time period of the specified day
    return genre_df_sorted[:10]

**Comparing top 10 genres of listened music for two cities on Monday and Friday**

Let's compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [30]:
# function call for Monday morning in Moscow (instead of df — moscow_general table)
# time objects are strings and are compared as strings
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [31]:
# function call for Monday morning in St. Petersburg (instead of df — spb_general table)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [32]:
# function call for Monday evening in Moscow
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [33]:
# function call for Monday evening in St. Petersburg
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Conclusions**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values occupy a significant proportion in the data and threaten the reliability of the research.

Friday evening does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning and at the end of the week.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, missing values in the data cast doubt on this result. There are so many gaps for Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

**Group data by listening genre for Moscow**

Let's group the `moscow_general` table by genre and count the listening of tracks of each genre using the `count()` method. Then we sort the result in descending order and store it in the `moscow_genres` table.

In [34]:
# in one line: grouping the moscow_general table by the 'genre' column,
# counting the number of 'genre' values in this group using the count() method,
# sort the resulting Series in descending order and save to moscow_genres
moscow_genres = moscow_general.groupby(by='genre')['genre'].count().sort_values(ascending=False)

**Displaying a sorted table by music genre for Moscow**

Let's display the first ten rows of `moscow_genres`:

In [35]:
# view the first 10 rows of moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

**Group data by listening genre for St. Petersburg**


Now let's repeat the same for St. Petersburg.

Let's group the `spb_general` table by genre. Let's count listening to tracks of each genre. Sort the result in descending order and store it in the `spb_genres` table:

In [36]:
# in one line: grouping the spb_general table by the 'genre' column,
# counting the number of 'genre' values in this group using the count() method,
# sort the resulting Series in descending order and save to spb_genres
spb_genres = spb_general.groupby(by='genre')['genre'].count().sort_values(ascending=False)

**Displaying a sorted table by music genre for St. Petersburg**

Let's display the first ten rows of `spb_genres`:

In [37]:
# view the first 10 rows of spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusions**

The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## Results of the research

We tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of Moscow and St. Petersburg users have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.

**In practice, researhing has to contain tests of statistical hypotheses.**
From the data of one service, it is not always possible to draw a conclusion about all the inhabitants of the city.
Tests of statistical hypotheses will show how reliable they are, based on the available data. But this question is beyond the scope of this research.